In [2]:
from selective_recruitment.plotting import plot_parcels
from selective_recruitment.plotting import plot_connectivity_weight

import selective_recruitment.globals as gl

from cortico_cereb_connectivity.scripts.script_summarize_weights import avrg_weight_map

import Functional_Fusion.atlas_map as am

import numpy as np
import SUITPy.flatmap as flatmap
import nitools as nt
import nibabel as nb
import matplotlib.colors as colors
from nilearn.plotting.cm import _cmap_d as nilearn_cmaps

# Choose a cerebellar region
For the motor task, the region of interest is M3R from Hierarchical parcellation and for the working memory task it's D3R

In [3]:
# get the plot of region of interest in the cerebellum
# create a colormap with two colors, red and blue
cmap = colors.ListedColormap(['#FFC20A'])

cerebellum_roi = "NettekovenSym32"
parcel_fig = plot_parcels(parcellation = cerebellum_roi, 
                   atlas_space = "SUIT3", 
                   roi_exp= "M.?3.?R",  # "D.?3.?R|D.?2.?R"
                   stats = "mode",
                   split = [2, 1, 2, 1,2, 1,2, 1,2, 1,2, 1, 2, 1, 2, 1], 
                   render = "plotly", 
                   cmap = cmap)
parcel_fig.show()
# parcel_fig.write_image("M3R.pdf")

['M3R']


Plot the whole atlas

In [ ]:
atlas, ainfo = am.get_atlas(atlas_str = "SUIT3", atlas_dir = gl.atlas_dir) 
fname = gl.atlas_dir + f'/{ainfo["dir"]}/atl-NettekovenSym32_space-SUIT_probseg.nii'
_, cmap, labels = nt.read_lut(gl.atlas_dir + f'/{ainfo["dir"]}/atl-NettekovenSym32.lut')
pseg = nb.load(fname)

surf_data = flatmap.vol_to_surf(pseg, stats='nanmean',
                                             space='SUIT')
label = np.argmax(surf_data, axis=1) + 1
ax = flatmap.plot(label,
    cmap=cmap,
    new_figure=False,
    label_names=labels,
    overlay_type='label',
    colorbar=False,
    render = "plotly",
    bordersize=1.5)
# plt.savefig(ut.figure_dir + f'NettekovenSym68_flatmap{backgroundcolor}.png', dpi=300,bbox_inches='tight')

In [ ]:
# plot the corresponding connectivity weights
cortex_roi = "Icosahedron1002" # glasser
method = "L2Regression"
dataset_name = "Fusion"
extension = "09"
ses_id = "all"

# get connectivity weight maps for the selected region
cii_img = avrg_weight_map(method = method,
                    cortex_roi = cortex_roi,
                    cerebellum_roi = "NettekovenSym32",
                    cerebellum_atlas = "SUIT3",
                    space="SUIT",
                    extension = extension,
                    dataset_name = dataset_name,
                    ses_id = ses_id,
                    train_t = 'train', 
                    type = 'scalar'
                    )


In [ ]:
roi_name = "D3R"
view = "medial"
cmap = "bwr"
colorbar = False
ax = plot_connectivity_weight(cii_img, 
                             roi_name = roi_name,
                             view = view,
                             cmap = cmap,
                             colorbar = colorbar, 
                             cscale = [-0.001, 0.001])


# setting up cameras to get a better view of the hand area in the lateral view
# rotate to get a better view of M1 only on lateral view
camera_params = []
camera_params.append(dict( #left hemi
    center=dict(x=0,y=0,z=0),
    eye=dict(x=-1.5, y=0,z=0.9),
    up=dict(x=0,y=0,z=1),
))
camera_params.append(dict( # right hemi
    center=dict(x=0,y=0,z=0),
    eye=dict(x=1.5, y=0,z=0.9),
    up=dict(x=0,y=0,z=1),
))
h = 0
hemis = ["L", "R"]
if view == "lateral":
    ax[h].update_layout(scene_camera=camera_params[h])
ax[h].show()
# ax[h].write_image(f"{roi_name}_{dataset_name}_{ses_id}_{extension}_{method}_avg_conn_weight_{roi_name}_{hemis[h]}_{view}.pdf")